# module import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm.notebook import tqdm
import random
import tensorflow as tf
from functools import partial
import glob
AUTOTUNE = tf.data.experimental.AUTOTUNE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)

from albumentations import (Compose, Normalize, RandomBrightness, HueSaturationValue, RandomContrast, HorizontalFlip,Rotate)

from keras.models import Model, Sequential
from keras.layers import *
from keras.initializers import glorot_uniform
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import DenseNet201, ResNet50, EfficientNetV2L, InceptionResNetV2, Xception, vgg19
from tensorflow.keras.applications.efficientnet import EfficientNetB3

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

Num GPUs Available:  1
2.14.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#unzip train/test dataset

- 국  
순대국_16.zip,
뼈해장국_17.zip,
선지해장국_17_19.zip
- 탕  
갈비탕_18.zip,
설렁탕_19.zip,
꽃개탕_19.zip,
알탕_20.zip,
닭볶음탕_20.zip,
도가니탕_20.zip,
- 찌개  
된장찌개_18.zip,
부대찌개_22.zip,
곱탕전골_22.zip

In [ ]:
%cd /content/drive/MyDrive/EWHA/Term Project/dataset/231028

/content/drive/.shortcut-targets-by-id/1I4hGaJ1S9nwYHdyRbFi5wqsEtoca4i7n/Term Project/dataset/231028


In [ ]:
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/순대국_16.zip" -d "/content/순대국"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/뼈해장국_17.zip" -d "/content/뼈해장국"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/선지해장국_17_19.zip" -d "/content/선지해장국"

!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/갈비탕_18.zip" -d "/content/갈비탕"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/설렁탕_19.zip" -d "/content/설렁탕"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/꽃게탕_19.zip" -d "/content/꽃게탕"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/알탕_20.zip" -d "/content/알탕"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/닭볶음탕_20.zip" -d "/content/닭볶음탕"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/도가니탕_20.zip" -d "/content/도가니탕"

!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/된장찌개_18.zip" -d "/content/된장찌개"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/부대찌개_22.zip" -d "/content/부대찌개"
!unzip "/content/drive/MyDrive/EWHA/Term Project/dataset/231028/곱창전골_22.zip" -d "/content/곱창전골"

Archive:  /content/drive/MyDrive/EWHA/Term Project/dataset/231028/순대국_16.zip
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160343483641765_0.jpeg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160353561629147_0 - Copy.jpeg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160361761541460_1.jpeg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160370774398681_0 - Copy.jpeg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160370865815740_1 - Copy.jpg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160413229977551_0.jpg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160431049729848_1 - Copy.jpg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160456222781021_0 - Copy.jpg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160463893911911_1 - Copy.jpg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160500452637322_1.jpeg  
  inflating: /content/순대국/╝°┤ы▒╣_16/04_041_04011010_160501249454007_0 - Copy.jpg  
  inflating: /content/순대국/╝°

속 분류 변경 필요(직접 수정!!)
- soup = ['뼈해장국/뼈해장국', '선지해장국/선지해장국', '순대국/순대국']
- tang = ['갈비탕', '꽃게탕', '닭볶음탕/닭볶음탕', '도가니탕/도가니탕', '설렁탕', '알탕/알탕']
- jjigae = ['곱창전골', '된장찌개', '부대찌개']

In [ ]:
import os
import shutil
# content 밑에 train 폴더 생성
%cd /content/
os.mkdir('train')
os.mkdir('test')

/content


*notice : 처음에 뼈해장국/뼈해장국으로 테스트하는 바람에 삭제한채로 진행함. 재실행시 경로 이것으로 수정후 아래 진행바람.

In [ ]:
def move_img_trte_split(base_dir, category, train_cnt, test_cnt):
    category_dir = os.path.join(base_dir, category)
    os.chdir(category_dir)
    images = [f for f in os.listdir() if f.lower().endswith('.jpg') or f.lower().endswith('.jpeg')]

    os.chdir(base_dir)
    train_images = images[:-10]  # 200개 이미지 중 10개는 test set (5%)
    test_images = images[-10:]

    prev_train_cnt = train_cnt
    prev_test_cnt = test_cnt

    for image in train_images:
        image_path = os.path.join(category_dir, image)
        new_path = os.path.join('train', f'TRAIN_{train_cnt}')
        shutil.move(image_path, new_path)
        train_cnt += 1

    for image in test_images:
        image_path = os.path.join(category_dir, image)
        new_path = os.path.join('test', f'TEST_{test_cnt}')
        shutil.move(image_path, new_path)
        test_cnt += 1

    print(f"{category}은 TRAIN_{prev_train_cnt} ~ TRAIN_{train_cnt - 1} 까지입니다.")
    print(f"{category}은 TEST_{prev_test_cnt} ~ TEST_{test_cnt - 1} 까지입니다.")

In [ ]:
base_directory = '/content/' ##
soup = ['뼈해장국/뼈해장국', '선지해장국/선지해장국', '순대국/순대국']
tang = ['갈비탕', '꽃게탕', '닭볶음탕/닭볶음탕', '도가니탕/도가니탕', '설렁탕', '알탕/알탕']
jjigae = ['곱창전골', '된장찌개', '부대찌개']
big_categories = [soup, tang, jjigae]

# train_cnt = 0
# test_cnt = 0
cnt = 0
for cat in big_categories:
  categories = cat # soup, tang, jjigae
  for _, category in enumerate(categories):
      train_cnt = cnt * 190
      test_cnt = cnt * 10
      move_img_trte_split(base_directory, category, train_cnt, test_cnt)
      cnt += 1

뼈해장국/뼈해장국은 TRAIN_0 ~ TRAIN_188 까지입니다.
뼈해장국/뼈해장국은 TEST_0 ~ TEST_9 까지입니다.
선지해장국/선지해장국은 TRAIN_190 ~ TRAIN_379 까지입니다.
선지해장국/선지해장국은 TEST_10 ~ TEST_19 까지입니다.
순대국/순대국은 TRAIN_380 ~ TRAIN_569 까지입니다.
순대국/순대국은 TEST_20 ~ TEST_29 까지입니다.
갈비탕은 TRAIN_570 ~ TRAIN_759 까지입니다.
갈비탕은 TEST_30 ~ TEST_39 까지입니다.
꽃게탕은 TRAIN_760 ~ TRAIN_949 까지입니다.
꽃게탕은 TEST_40 ~ TEST_49 까지입니다.
닭볶음탕/닭볶음탕은 TRAIN_950 ~ TRAIN_1139 까지입니다.
닭볶음탕/닭볶음탕은 TEST_50 ~ TEST_59 까지입니다.
도가니탕/도가니탕은 TRAIN_1140 ~ TRAIN_1329 까지입니다.
도가니탕/도가니탕은 TEST_60 ~ TEST_69 까지입니다.
설렁탕은 TRAIN_1330 ~ TRAIN_1519 까지입니다.
설렁탕은 TEST_70 ~ TEST_79 까지입니다.
알탕/알탕은 TRAIN_1520 ~ TRAIN_1709 까지입니다.
알탕/알탕은 TEST_80 ~ TEST_89 까지입니다.
곱창전골은 TRAIN_1710 ~ TRAIN_1899 까지입니다.
곱창전골은 TEST_90 ~ TEST_99 까지입니다.
된장찌개은 TRAIN_1900 ~ TRAIN_2089 까지입니다.
된장찌개은 TEST_100 ~ TEST_109 까지입니다.
부대찌개은 TRAIN_2090 ~ TRAIN_2279 까지입니다.
부대찌개은 TEST_110 ~ TEST_119 까지입니다.


In [ ]:
# # soup = ['뼈해장국/뼈해장국', '선지해장국/선지해장국', '순대국/순대국']
# %cd '/content/뼈해장국/뼈해장국/'
# #os.listdir() ; 뼈해장국의 파일들
# images = [f for f in os.listdir() if '.jpg' or '.jpeg' in f.lower()]

# %cd '/content/'
# train_images = images[:-10] #200개 이미지 중 10개는 test set (5%)
# test_images = images[-10:]

# train_cnt = 0
# test_cnt = 0
# for image in train_images:
#     image = '뼈해장국/뼈해장국/' + image
#     new_path = 'train/TRAIN_' + str(train_cnt)
#     shutil.move(image, new_path)
#     train_cnt += 1

# for image in test_images:
#     image = '뼈해장국/뼈해장국/' + image
#     new_path = 'test/TEST_' + str(test_cnt)
#     shutil.move(image, new_path)
#     test_cnt += 1
# print("[국] 뼈해장국은 TRAIN_{} ~ TRAIN_{} 까지입니다.".format(0, train_cnt - 1))
# print("[국] 뼈해장국은 TEST_{} ~ TEST_{} 까지입니다.".format(0, test_cnt - 1))
# ##############################################################################################################

# %cd '/content/선지해장국/선지해장국/'
# images = [f for f in os.listdir() if '.jpg' or '.jpeg' in f.lower()]

# %cd '/content/'
# train_images = images[:-10] #200개 이미지 중 10개는 test set (5%)
# test_images = images[-10:]

# prev_train_cnt = train_cnt
# prev_test_cnt = test_cnt

# for image in train_images:
#     image = '선지해장국/선지해장국/' + image
#     new_path = 'train/TRAIN_' + str(train_cnt)
#     shutil.move(image, new_path)
#     train_cnt += 1

# for image in test_images:
#     image = '선지해장국/선지해장국/' + image
#     new_path = 'test/TEST_' + str(test_cnt)
#     shutil.move(image, new_path)
#     test_cnt += 1

# print("[국] 선지해장국은 TRAIN_{} ~ TRAIN_{} 까지입니다.".format(prev_train_cnt, train_cnt - 1))
# print("[국] 선지해장국은 TEST_{} ~ TEST_{} 까지입니다.".format(prev_test_cnt, test_cnt - 1))
# ##############################################################################################################
# %cd '/content/순대국/순대국/'
# images = [f for f in os.listdir() if '.jpg' or '.jpeg' in f.lower()]

# %cd '/content/'
# train_images = images[:-10] #200개 이미지 중 10개는 test set (5%)
# test_images = images[-10:]

# prev_train_cnt = train_cnt
# prev_test_cnt = test_cnt

# for image in train_images:
#     image = '순대국/순대국/' + image
#     new_path = 'train/TRAIN_' + str(train_cnt)
#     shutil.move(image, new_path)
#     train_cnt += 1

# for image in test_images:
#     image = '순대국/순대국/' + image
#     new_path = 'test/TEST_' + str(test_cnt)
#     shutil.move(image, new_path)
#     test_cnt += 1

# print("[국] 순대국은 TRAIN_{} ~ TRAIN_{} 까지입니다.".format(prev_train_cnt, train_cnt - 1))
# print("[국] 순대국은 TEST_{} ~ TEST_{} 까지입니다.".format(prev_test_cnt, test_cnt - 1))

In [ ]:
!zip -r train.zip /content/train

  adding: content/train/ (stored 0%)
  adding: content/train/TRAIN_678 (deflated 0%)
  adding: content/train/TRAIN_2201 (deflated 0%)
  adding: content/train/TRAIN_409 (deflated 0%)
  adding: content/train/TRAIN_1037 (deflated 0%)
  adding: content/train/TRAIN_987 (deflated 1%)
  adding: content/train/TRAIN_270 (deflated 0%)
  adding: content/train/TRAIN_1261 (deflated 3%)
  adding: content/train/TRAIN_965 (deflated 0%)
  adding: content/train/TRAIN_441 (deflated 1%)
  adding: content/train/TRAIN_2004 (deflated 0%)
  adding: content/train/TRAIN_1820 (deflated 0%)
  adding: content/train/TRAIN_1109 (deflated 0%)
  adding: content/train/TRAIN_583 (deflated 0%)
  adding: content/train/TRAIN_1775 (deflated 0%)
  adding: content/train/TRAIN_474 (deflated 0%)
  adding: content/train/TRAIN_793 (deflated 0%)
  adding: content/train/TRAIN_131 (deflated 1%)
  adding: content/train/TRAIN_484 (deflated 0%)
  adding: content/train/TRAIN_455 (deflated 0%)
  adding: content/train/TRAIN_141 (deflated 

In [ ]:
!zip -r test.zip /content/test

  adding: content/test/ (stored 0%)
  adding: content/test/TEST_44 (deflated 0%)
  adding: content/test/TEST_5 (deflated 0%)
  adding: content/test/TEST_23 (deflated 0%)
  adding: content/test/TEST_41 (deflated 0%)
  adding: content/test/TEST_64 (deflated 1%)
  adding: content/test/TEST_110 (deflated 0%)
  adding: content/test/TEST_97 (deflated 0%)
  adding: content/test/TEST_33 (deflated 1%)
  adding: content/test/TEST_45 (deflated 0%)
  adding: content/test/TEST_7 (deflated 0%)
  adding: content/test/TEST_78 (deflated 1%)
  adding: content/test/TEST_87 (deflated 1%)
  adding: content/test/TEST_72 (deflated 3%)
  adding: content/test/TEST_86 (deflated 2%)
  adding: content/test/TEST_85 (deflated 0%)
  adding: content/test/TEST_77 (deflated 0%)
  adding: content/test/TEST_49 (deflated 0%)
  adding: content/test/TEST_71 (deflated 3%)
  adding: content/test/TEST_24 (deflated 1%)
  adding: content/test/TEST_83 (deflated 1%)
  adding: content/test/TEST_27 (deflated 0%)
  adding: content/te

In [ ]:
from google.colab import files
files.download('/content/test.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/train.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.move('/content/test.zip', '/content/drive/MyDrive/EWHA/test.zip')

'/content/drive/MyDrive/EWHA/test.zip'

In [ ]:
shutil.move('/content/train.zip', '/content/drive/MyDrive/EWHA/train.zip')

'/content/drive/MyDrive/EWHA/train.zip'

In [ ]:
%cd /content/drive/MyDrive/EWHA/
%ls

/content/drive/MyDrive/EWHA
 1104_datapreprocessing.ipynb       EPITA/               PatternRecognition/   train.zip
 20231025_datapreprocessing.ipynb   Machine_Learning/   'Term Project'@
 DataScience/                       only_movedir.ipynb   test.zip
